In [1]:
import numpy as np
import pandas as pd
# from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import copy
import math
import pickle
from sklearn.metrics import roc_auc_score
import optuna
import catboost as cb
import warnings
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
warnings.filterwarnings('ignore')
from tqdm.auto import tqdm

In [2]:
train_df = pd.read_parquet("train_data.pqt")
test_df = pd.read_parquet("test_df.pqt")

In [3]:
test_df['start_cluster'] = test_df['start_cluster'].fillna(method='ffill')

In [4]:
grouped_df_first = test_df.groupby('id').first().reset_index()
merged_df = pd.merge(test_df, grouped_df_first, on='id', suffixes=('', '_first'))

grouped_df_second = test_df.groupby('id').nth(1).reset_index()
merged_df = pd.merge(merged_df, grouped_df_second, on='id', suffixes=('', '_second'))

merged_df = merged_df.loc[:, ~merged_df.columns.duplicated()]

In [5]:
test_df = merged_df[merged_df['date'] == 'month_6']

In [6]:
grouped_df_first = train_df.groupby('id').first().reset_index()
merged_df = pd.merge(train_df, grouped_df_first, on='id', suffixes=('', '_first'))

grouped_df_second = train_df.groupby('id').nth(1).reset_index()
merged_df = pd.merge(merged_df, grouped_df_second, on='id', suffixes=('', '_second'))

merged_df = merged_df.loc[:, ~merged_df.columns.duplicated()]

In [7]:
train_df = merged_df[merged_df['date'] == 'month_3']

In [8]:
corr_matrix = train_df.corr().abs()

# Получение верхнего треугольника матрицы корреляции (без диагонали)
upper_triangle = corr_matrix.where(
    np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

# Нахождение колонок, где корреляция больше 0.9
to_drop = [column for column in upper_triangle.columns if any(upper_triangle[column] > 0.95)]

train_df =train_df.drop(to_drop, axis=1)


In [9]:
test_df =test_df.drop(to_drop, axis=1)

In [10]:
category_columns = ['start_cluster', 'channel_code', 'city', 'city_type', 
                    'index_city_code', 'ogrn_month', 'ogrn_year', 'okved', 'segment']

In [11]:
def clever_one_hot(df, col): #переписать в lable encoding?
    top_4 = df[col].value_counts().index[:4]
    df.loc[~df[col].isin(top_4), col] = 'other'
    one_hot_encoded = pd.get_dummies(df[col], prefix=col)
    df = df.drop(col, axis=1)
    return pd.concat([df, one_hot_encoded], axis=1)

def feature_prossesing(df):
    one_hot_encoded = pd.get_dummies(df['segment'], prefix='seg')
    df = df.drop('segment', axis=1)
    df = pd.concat([df, one_hot_encoded], axis=1)

    df['ogrn_year'] = df['ogrn_year'].fillna('_-1')
    df['ogrn_month'] = df['ogrn_month'].fillna('_-1')
    df['ogrn_year'] = df['ogrn_year'].apply(lambda x: int(x.split('_')[-1]))
    df['ogrn_month'] = df['ogrn_month'].apply(lambda x: int(x.split('_')[-1]))
    df['lasting'] = df['ogrn_year']*12 + df['ogrn_month']
    df['lasting'] = df['lasting'].apply(lambda x: x if x>=0 else -1)

    del df['ogrn_year']
    del df['ogrn_month']

    for i in ['channel_code', 'city_type']:
        df = clever_one_hot(df, i)

    label_encoder = LabelEncoder()
    df['start_cluster'] = label_encoder.fit_transform(df['start_cluster'])
    df['end_cluster'] = label_encoder.transform(df['end_cluster'])
    category_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))

    df = df.drop(df.select_dtypes(include=['object']).columns, axis=1)
    df = df.fillna(-1)

    return df, category_mapping   

In [12]:
df, category_mapping = feature_prossesing(train_df)

In [13]:
def clever_one_hot(df, col, df1): #переписать в lable encoding?
    top_4 = df1[col].value_counts().index[:4]
    df.loc[~df[col].isin(top_4), col] = 'other'
    one_hot_encoded = pd.get_dummies(df[col], prefix=col)
    df = df.drop(col, axis=1)
    return pd.concat([df, one_hot_encoded], axis=1)

def feature_prossesing(df, df_1):
    one_hot_encoded = pd.get_dummies(df['segment'], prefix='seg')
    df = df.drop('segment', axis=1)
    df = pd.concat([df, one_hot_encoded], axis=1)

    df['ogrn_year'] = df['ogrn_year'].fillna('_-1')
    df['ogrn_month'] = df['ogrn_month'].fillna('_-1')
    df['ogrn_year'] = df['ogrn_year'].apply(lambda x: int(x.split('_')[-1]))
    df['ogrn_month'] = df['ogrn_month'].apply(lambda x: int(x.split('_')[-1]))
    df['lasting'] = df['ogrn_year']*12 + df['ogrn_month']
    df['lasting'] = df['lasting'].apply(lambda x: x if x>=0 else -1)

    del df['ogrn_year']
    del df['ogrn_month']

    for i in ['channel_code', 'city_type']:
        df = clever_one_hot(df, i, df_1)

    label_encoder = LabelEncoder()
    df['start_cluster'] = label_encoder.fit_transform(df['start_cluster'])
    #df['end_cluster'] = label_encoder.transform(df['end_cluster'])
    category_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))

    df = df.drop(df.select_dtypes(include=['object']).columns, axis=1)
    df = df.fillna(-1)

    return df, category_mapping   

In [14]:
test, category_mapping = feature_prossesing(test_df, train_df)

Модели

табнет

In [17]:
# !pip install pytorch-tabnet

In [25]:
y = df['end_cluster']
X = df.drop(['end_cluster'], axis=1)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=42)
X_train = X_train.to_numpy()
y_train = y_train.to_numpy().squeeze()
X_val = X_val.to_numpy()
y_val = y_val.to_numpy().squeeze()

X_test  = test.to_numpy()

In [48]:
y_train_onehot = np.eye(17)[y_train]
y_val_onehot = np.eye(17)[y_val]

In [21]:
from pytorch_tabnet.multitask import TabNetMultiTaskClassifier
import torch

In [50]:
clf = TabNetMultiTaskClassifier(
                                n_steps=17,
                                # cat_idxs=cat_idxs,
                                # cat_dims=cat_dims,
                                # cat_emb_dim=1,
                                optimizer_fn=torch.optim.Adam,
                                optimizer_params=dict(lr=2e-2),
                                scheduler_params={"step_size":50, # how to use learning rate scheduler
                                                  "gamma":0.9},
                                scheduler_fn=torch.optim.lr_scheduler.StepLR,
                                mask_type='entmax', # "sparsemax",
                                lambda_sparse=0, # don't penalize for sparser attention
                       
                      )

In [ ]:
max_epochs = 1000
clf.fit(
    X_train=X_train, y_train=y_train_onehot,
    eval_set=[(X_val,y_val_onehot)],
    max_epochs=max_epochs,
    patience=50, # please be patient ^^
    batch_size=1024,
    virtual_batch_size=128,
    num_workers=1,
    drop_last=False,
)

In [ ]:
predictions = classifier.predict_proba(X_test)